<a href="https://colab.research.google.com/github/PushyamiB/PushyamiB/blob/circleci-project-setup/RuleEngine_Symplfy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
def post_list_dict(url, list_dict):
    return requests.post(url, data=list_dict, headers={'Content-Type': 'application/json'}).content.decode()
def getJobIDDetails(jobId):
  data='[{"columnName": "sno","columnValue": "'+str(jobId)+'"}]'
  outputStr = post_list_dict('https://ezeeapi.meanhost.in/v1/api/Ezee_Reasons_Rejections/filter?andOr=AND&page=1&itemsPerPage=100',data)
  #     print(outputStr.content)
  diction = json.loads(outputStr)
  print(diction)
  return diction



In [ ]:
data='[{"columnName": "sno","columnValue": "'+str(134)+'"}]'
outputStr = post_list_dict('https://ezeeapi.meanhost.in/v1/api/Ezee_Reasons_Rejections/filter?andOr=AND&page=1&itemsPerPage=100',data)
#     print(outputStr.content)
diction = json.loads(outputStr)
def returnMismatch(dict_data):
  json_data = json.loads(dict_data['document']['records'][0]['jsondata'])
  GCC_Match = False
  Chassis_Match = False
  for i in json_data:
    if('field' in i and i['field']=='GCC_spec_img'):
      if('ismatched' in i and i['ismatched']=="Matched"):
        GCC_Match=True
    if('field' in i and i['field']=='ChassisNo_img'):
      if('ismatched' in i and i['ismatched']=="Matched"):
        Chassis_Match=True
    finalBool = GCC_Match and Chassis_Match
  return finalBool

In [ ]:
# def populateListFromJobID(jobId):
#     dict_data=getJobIDDetails(jobId)
#     dict_dmg=[]
#     try:
#       json_data = diction['document']['records'][0]['jsondata']
#     except Exception as e:
#       print("No json Data present in records")
#       return dict_dmg
#     for i in range(len(json_data)):
#       if 'Aniitationlist' in json_data[i]:
#         for j in range(len(json_data[i]['Aniitationlist'])):
#           dict_dmg.append({"body_part":json_data[i]['Aniitationlist'][j]['body_part'][0]['id'],"damage_type":json_data[i]['Aniitationlist'][j]['damage_type'][0]['id'],"damage_serverity":json_data[i]['Aniitationlist'][j]['damage_serverity'][0]['id']}) 
#         #print(dict_dmg)
#     return dict_dmg   


In [ ]:
def populateListFromDict(dict_data):
    dict_dmg=[]
    json_data = json.loads(dict_data['document']['records'][0]['jsondata'])
    for i in range(len(json_data)):
      # print(json_data[i])
      serializeObj(json_data[i])
      if 'Aniitationlist' in json_data[i]:
        for j in range(len(json_data[i]['Aniitationlist'])):
          try:
            dict_dmg.append({"body_part":json_data[i]['Aniitationlist'][j]['body_part'][0]['id'],"damage_type":json_data[i]['Aniitationlist'][j]['damage_type'][0]['id'],"damage_serverity":json_data[i]['Aniitationlist'][j]['damage_serverity'][0]['id']}) 
          except:
            print("None present : Invalid format , Ignoring current Annotation input")
        #print(dict_dmg)
        ##If empty list recomended
    return dict_dmg   

In [ ]:
flaskRuleEng(153)

NameError: ignored

In [ ]:
json_d = extractObj('roi.obj')
json_d

In [ ]:
import pickle
def serializeObj(obj):
    with open('roi.obj', 'wb') as roi_freader:
      pickle.dump(obj, roi_freader)
def extractObj(file_path):
  pkl_file= open(file_path, 'rb')
  obj = pickle.load(pkl_file)
  return obj

In [ ]:
def ruleEng(df):
    ws_df = df[(df["body_part"]=="Wind Shield Front")|(df["body_part"]=="Wind Shield Rear")] 
    # if(len(df[df["Body part"]=="Wind Shield Front"])>0 or len(df[df["Body part"]=="Wind Shield Rear"])>0):
    # len(ws_df)
    if(len(ws_df)>0):
      # print("True")
      if((len(ws_df[ws_df["damage_type"]=="Cracked"])>0) | (len(ws_df[ws_df["damage_type"]=="Broken"])>0)):
        return False
    #vh_df = df[df["vehType"]=="Passenger"]
    #print(vh_df)
    vh_df = df
    # serializeObj(df)
    total_df=df.groupby(['damage_type','body_part']).size()
    display(total_df)
    # df_2=total_df['damage_serverity']
    
    # damage_df = total_df['damage_severity']
    # vh_sc_df = damage_df.loc['Scratched']
    # vh_dnt_df = damage_df.loc['Dent Small']
    # vh_Dnt_df = damage_df.loc['Dent Large']
    # vh_dnt_df = total_df[total_df['damage_serverity']=='Dent Small']['damage_severity']
    # vh_Dnt_df = total_df[total_df['damage_serverity']=='Dent Large']['damage_severity']

    if("Scratched" in total_df and len(total_df['Scratched'])>2):
      return False
    elif("Dent Small" in total_df and len(total_df['Dent Small'])>2):
      return False
    elif ("Dent Large" in total_df and len(total_df['Dent Large'])>1):
      return False
    elif ("Tear" in total_df):
      return False
    else:
      return True


In [ ]:
def flaskRuleEng(jobID):
  diction = getJobIDDetails(jobID)
  #Checking if json data exists
  try:
    json_data = json.loads(diction['document']['records'][0]['jsondata'])
  except Exception as e:
    print("No json Data present in records")
    return {"Recomended":0} ## Not recomended
  dict_dmg=populateListFromDict(diction)
  
  # if(not returnMismatch(diction)):
  #   print("Not matching GCC or Chassis number")
  #   return {"Recomended":0}
  if len(dict_dmg)==0:
    print("No damages!")
    return {"Recomendded":1}

  print("Data:" ,dict_dmg)
  df_dmg=pd.DataFrame(dict_dmg)
  # df_dmg = df_dmg.append({'body_part': 'Hood', 'damage_type': 'Scratched', 'damage_serverity': 'Medium'},ignore_index=True)
  # df_dmg = df_dmg.append({'body_part': 'Hood', 'damage_type': 'Dent Small', 'damage_serverity': 'Medium'},ignore_index=True)
  display(df_dmg)
  output= ruleEng(df_dmg)
  if output:
    return {"Recomended":1}
  return {"Recomended":0}


In [ ]:
import pickle
import pandas as pd
# pkl_file= open('roi.obj', 'rb')
# df = pickle.load(pkl_file)
df=pd.DataFrame()
df = df.append({'body_part': 'Hood', 'damage_type': 'Dent', 'damage_serverity': 'Medium'},ignore_index=True)
# df = df.append({'body_part': 'Hood', 'damage_type': 'Tear', 'damage_serverity': 'Medium'},ignore_index=True)
# df = df[['body_part','damage_type']]

df

,body_part,damage_serverity,damage_type
0,Hood,Medium,Dent


In [ ]:
ruleEng(df)

damage_type  body_part
Dent         Hood         1
dtype: int64

True

In [ ]:
df_sc = df.groupby(['damage_type','body_part']).size()
# df_sc = df_sc.groupby(['damage_type']).sum()
# df_sc['damage_serverity']['Scratched']
# df_sc.loc['damage_type']
# df.groupby('damage_type').filter(lambda group: group.size > 2)
''
# len(df_sc['Dent'])
# len(df_sc['Scratched'])
'Dent Small' in df_sc

In [ ]:
df_3=df['damage_serverity']
df_3.loc["Scratched"]

In [ ]:
json_data = extractObj("roi.obj")
json_data

In [ ]:
flaskRuleEng(19)

In [ ]:
vh_df = df[df["vehType"]=="Passenger"]
agg = vh_df.groupby(by=["Damage","Body part"]).count()
total_df=agg.groupby('Damage').sum()


In [ ]:
def insertfinalResultInDatabase(jobID,pushData):
#     data = {"jsondata": repr(listObj)}
    url = 'https://ezeeapi.meanhost.in/v1/api/Ezee_Reasons_Rejections/'+str(jobID)
    headers = {'Content-type': 'application/json'}
#     print(pushData)
    pushData = json.dumps(pushData).replace('True','true')
    r = requests.post(url, data=pushData, headers=headers)
    print(r.content.decode())

In [ ]:
json_data={'0': {
        "Body part": "Bumper Front Right Hand Side",
        "Damage": "Scratch",
        "vehType": "Commercial"
    },'1': {
        "Body part": "Bumper Front Left Hand Side",
        "Damage": "Scratch",
        "vehType": "Passenger"
    },'3': {
        "Body part": "Bumper Front Right Hand Side",
        "Damage": "Scratch",
        "vehType": "Commercial"
    },'4': {
        "Body part": "Bumper Front Right Hand Side",
        "Damage": "Dent",
        "vehType": "Passenger"
    },'5': {
        "Body part": "Wind Shield Rear",
        "Damage": "Scratch",
        "vehType": "Passenger"
    }
}

In [ ]:
panel_count,damage_count=countFrequency(json_data)
print(panel_count , damage_count)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(json_data.values()))
k = df.groupby(by=["Body part","Damage","vehType"]).size()
ws_df = df[df["Body part"]=="Wind Shield Rear"]
k

In [ ]:
k = df.groupby(by=["Damage","Body part","vehType"]).count()
k

In [ ]:
df

In [ ]:
vh_df = df[df["vehType"]=="Passenger"]
agg = vh_df.groupby(by=["Damage","Body part"]).count()
agg.groupby('Damage').sum()


In [ ]:
df_dmg

In [ ]:
from numpy.core.numeric import False_
def ruleEng_v1(df):
    ws_df = df[(df["body_part"]=="Wind Shield Front")|(df["body_part"]=="Wind Shield Rear")] 
    # if(len(df[df["Body part"]=="Wind Shield Front"])>0 or len(df[df["Body part"]=="Wind Shield Rear"])>0):
    len(ws_df)
    if(len(ws_df)>0):
      # print("True")
      if(len(ws_df[ws_df["damage_type"]=="Scratch"])>0):
        return True
    #vh_df = df[df["vehType"]=="Passenger"]
    #print(vh_df)
    if df["vehType"]=="Passenger":
      vh_df = df[df["vehType"]=="Passenger"]
      vh_sc_df = vh_df[vh_df['damage_type']=='Scratch']
      vh_dnt_df = vh_df[vh_df['damage_type']=='Dent_Small']
      vh_Dnt_df = vh_df[vh_df['damage_type']=='Dent_Large']
      if(len(vh_sc_df)>3):
        return False
      elif(len(vh_dnt_df)>2):
        return False
      elif(len(vh_Dnt_df)>1):
        return False
    else:
      return True
    if df["vehType"]=="Commercial": 
      cm_df = df[df["vehType"]=="Commercial"]
      cm_sc_df = cm_df[cm_df['Damage']=='Scratch']
      cm_dnt_df = cm_df[cm_df['Damage']=='Dent_Small']
      cm_Dnt_df = cm_df[cm_df['Damage']=='Dent_Large']
      if(len(cm_sc_df)>3):
        return False
      elif(len(cm_dnt_df)>3):
        return False
      elif(len(cm_Dnt_df)>2):
        return False
      else:
        return True  
         

In [ ]:
def ruleEngine(vehType , bodyPanel, damage_json):
  if(bodyPanel == "Wind Shield Front" or bodyPanel == "Wind Shield Rear"):
    if("Scratch" in damage_json):
      return True
  else:
      if(vehType=="Passenger"):
        if("Scratch" in damage_json and damage_json["Scratch"]> 3):
          return True
        elif("Dent_Small" in damage_json and damage_json["Dent_Small"]> 2):
          return True
        elif("Dent_Large" in damage_json and damage_json["Dent_Large"]> 1):
          return True
      else:
        if("Scratch" in damage_json and damage_json["Scratch"]> 3):
          return True
        elif("Dent_Small" in damage_json and damage_json["Dent_Small"]> 2):
          return True
        elif("Dent_Large" in damage_json and damage_json["Dent_Large"]> 2):
          return True
  return False


In [ ]:
def countFrequency(json_data):
    freq_part= {}
    windShield_dict = {}
    for item in json_data.items():
          if (item[1]['Damage'] in freq_part):
            # print('Output',freq_part[item[1]['Body part']])
            pass   
          else:
              freq_part[item[1]['Damage']]={}
          # if("Wind Shield Front" in item[1]['Body part'] or "Wind Shield Rear" in item[1]['Body part']):
          #   windShield_dict[]
          if(item[1]['Body part'] in freq_part[item[1]['Damage']]):
                freq_part[item[1]['Damage']][item[1]['Body part']] += 1
                
          else:
            freq_part[item[1]['Damage']][item[1]['Body part']]= 1 

    print(freq_part)
    diction = {}
    for k,v in freq_part.items(): 
      diction[k]=len(v.items())
    print(diction)
    return freq_part,diction

In [ ]:


freq_part= {}
for item in json_data.items():
      if (item[1]['Body part'] in freq_part):
         # print('Output',freq_part[item[1]['Body part']])
         pass   
      else:
          freq_part[item[1]['Body part']]={}
      if(item[1]['Damage'] in freq_part[item[1]['Body part']]):
            #print(freq_part[item[1]['Body part']['Damage']])
            freq_part[item[1]['Body part']][item[1]['Damage']] += 1
            #print(freq_part[item[1]['Body part']][item[1]['Damage']])
      else:
        freq_part[item[1]['Body part']][item[1]['Damage']]= 1    
          #print(freq_part)
#print(freq_part[item[1]['Body part']])     
print(freq_part)     
# for key, value in freq_part.items():
#     print ("% s : % s"%(key, value))

In [ ]:
def CountFrequency(my_list):
    freq_damage = {}
    for item in my_list:
        if (item in freq_damage):
            freq_damage[item] += 1
        else:
            freq_damage[item] = 1
    for key, value in freq_damage.items():
        print ("% s : % s"%(key, value))